In [1]:
import torch
import os
import shutil
import pandas as pd
import numpy as np
import talib

# --- 제공해주신 코드의 클래스들을 임포트합니다 ---

# 모델 관련
from models.model import (GATree, GATreePop, FEATURE_NUM, FEATURE_PAIR, ALL_FEATURES,
                   COL_NODE_TYPE, NODE_TYPE_UNUSED)

# 변이 연산자 관련
from evolution.Mutation.base import BaseMutation
from evolution.Mutation.chain import ChainMutation
from evolution.Mutation.node_param import NodeParamMutation
from evolution.Mutation.reinitialize_node import ReinitializeNodeMutation
from evolution.Mutation.add_node import AddNodeMutation
from evolution.Mutation.delete_node import DeleteNodeMutation
from evolution.Mutation.add_subtree import AddSubtreeMutation
from evolution.Mutation.delete_subtree import DeleteSubtreeMutation

# import data.data_download as data_dd
# import data.feature_generator as data_fe
# import data.candlestick_patterns as data_cp

import data.ta_lib_feature_generator as talib_feat

In [2]:
# df = data_dd.fetch_historical_data('BTCUSDT', '1m', 14)

df = pd.read_csv('dataset/btc_1m_t.csv')
df['Close time'] = pd.to_datetime(df['Close time'])
df = df[['Open', 'High', 'Low', 'Close', 'Volume', 'Close time']]

In [4]:
# ==============================================================================
# Feature Generation
# ==============================================================================
print("Generating technical analysis features...")

# 1. 추세 (Trend) 지표
print("- Calculating Trend Indicators...")
df, price_col = talib_feat.calculate_price_change_features(df)
df, ma_col = talib_feat.calculate_ma(df, window=14)
df, ema_col = talib_feat.calculate_ema(df, window=24)
df, vwma_col = talib_feat.calculate_vwma(df, window=5) # TA-Lib 미지원
df, macd_cols = talib_feat.calculate_macd(df, short_window=12, long_window=26, signal_window=9)
df, adx_cols = talib_feat.calculate_adx(df, window=14)
df, ichimoku_cols = talib_feat.calculate_ichimoku(df) # TA-Lib 미지원
# --- 추가된 추세 지표 ---
df, _ = talib_feat.calculate_dema(df, window=30)
df, _ = talib_feat.calculate_tema(df, window=30)
df, _ = talib_feat.calculate_sar(df, acceleration=0.02, maximum=0.2)
df, _ = talib_feat.calculate_trima(df, window=30)

# 2. 모멘텀 (Momentum) 지표
print("- Calculating Momentum Indicators...")
df, rsi_col = talib_feat.calculate_rsi(df, window=14)
df, stoch_cols = talib_feat.calculate_stochastic_oscillator(df, k_window=14, d_window=3)
df, williams_r_col = talib_feat.calculate_williams_r(df, window=14)
df, cci_col = talib_feat.calculate_cci(df, window=20)
# --- 추가된 모멘텀 지표 ---
df, _ = talib_feat.calculate_stochrsi(df, window=14, k_window=5, d_window=3)
df, _ = talib_feat.calculate_ppo(df, fast_window=12, slow_window=26)
df, _ = talib_feat.calculate_roc(df, window=10)
df, _ = talib_feat.calculate_ultosc(df, window1=7, window2=14, window3=28)
df, _ = talib_feat.calculate_mom(df, window=10)


# 3. 변동성 (Volatility) 지표
print("- Calculating Volatility Indicators...")
df, bb_cols = talib_feat.calculate_bollinger_bands(df, window=20, num_std=2)
df, atr_col = talib_feat.calculate_atr(df, window=14)
# --- 추가된 변동성 지표 ---
df, _ = talib_feat.calculate_natr(df, window=14)


# 4. 거래량 (Volume) 지표
print("- Calculating Volume Indicators...")
df, obv_col = talib_feat.calculate_obv(df)
df, cmf_col = talib_feat.calculate_cmf(df, window=20) # TA-Lib 미지원
# --- 추가된 거래량 지표 ---
df, _ = talib_feat.calculate_ad(df)
df, _ = talib_feat.calculate_adosc(df, fast_window=3, slow_window=10)


# 5. 주기 (Cycle) 지표
print("- Calculating Cycle Indicators...")
# --- 추가된 주기 지표 ---
df, _ = talib_feat.calculate_ht_dcperiod(df)
df, _ = talib_feat.calculate_ht_trendmode(df)
df, _ = talib_feat.calculate_ht_sine(df)


# 6. 지지 (Support) 및 저항 (Resistance) 지표
print("- Calculating Support & Resistance Indicators...")
df, sr_cols = talib_feat.calculate_support_resistance(df, window=14)


# 7. 캔들스틱 패턴 (함수 내부에 신규 패턴이 이미 추가됨)
print("- Calculating Candlestick Patterns...")
df, candle_cols = talib_feat.calculate_all_candlestick_patterns(df)


# ==============================================================================
# 최종 데이터 처리
# ==============================================================================
print("\nFinalizing data...")
# 결측치 처리 (TA-Lib 계산 시 초반 데이터는 NaN이 됨)
initial_rows = len(df)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
final_rows = len(df)

print(f"Feature generation complete. Dropped {initial_rows - final_rows} rows due to NaN values.")
print(f"Final DataFrame shape: {df.shape}")

Generating technical analysis features...
- Calculating Trend Indicators...
- Calculating Momentum Indicators...
- Calculating Volatility Indicators...
- Calculating Volume Indicators...
- Calculating Cycle Indicators...
- Calculating Support & Resistance Indicators...
- Calculating Candlestick Patterns...

Finalizing data...
Feature generation complete. Dropped 113 rows due to NaN values.
Final DataFrame shape: (20187, 90)


In [5]:
df

,Open,High,Low,Close,Volume,Close time,close_change_pct,body_size,upper_wick_size,lower_wick_size,...,ROC_10,ULTOSC_7_14_28,MOM_10,NATR_14,AD,ADOSC_3_10,HT_DCPERIOD,HT_TRENDMODE,HT_SINE,HT_LEADSINE
0,111155.37,111180.00,111155.37,111179.99,3.67785,2025-07-10 02:44:59.999,0.022149,24.62,0.01,0.00,...,-0.053946,32.659400,-60.01,0.027679,-34.684876,-5.572584,25.911186,0,-0.344449,0.420274
1,111180.00,111180.00,111150.00,111160.63,8.23964,2025-07-10 02:45:59.999,-0.017413,19.37,0.00,10.63,...,-0.099478,30.759076,-110.69,0.027634,-37.085358,-5.299080,26.302811,0,-0.243271,0.513846
2,111160.63,111166.00,111160.63,111166.00,7.37209,2025-07-10 02:46:59.999,0.004831,5.37,0.00,0.00,...,-0.043097,33.691795,-47.93,0.026004,-29.713268,-2.359792,26.821555,0,-0.142020,0.599516
3,111166.00,111225.04,111165.99,111225.03,7.74851,2025-07-10 02:47:59.999,0.053101,59.03,0.01,0.01,...,0.017553,51.267657,19.52,0.027926,-21.967382,1.521770,27.488377,0,-0.031174,0.684719
4,111225.03,111225.04,111225.03,111225.04,1.11452,2025-07-10 02:48:59.999,0.000009,0.01,0.00,0.00,...,0.022518,53.527060,25.04,0.025932,-20.852862,3.325959,28.205247,0,0.086555,0.765657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20182,118996.49,118996.49,118963.84,118987.04,8.28228,2025-07-24 03:06:59.999,-0.007941,9.45,0.00,23.20,...,0.005934,50.840630,7.06,0.024606,4911.481761,1.495723,20.162824,0,0.950856,0.891301
20183,118987.05,118987.05,118948.94,118948.94,7.75390,2025-07-24 03:07:59.999,-0.032020,38.11,0.00,0.00,...,-0.026097,49.579470,-31.05,0.025144,4903.727861,-0.674328,19.781956,0,0.997077,0.759064
20184,118948.94,118948.94,118894.59,118942.61,5.10386,2025-07-24 03:08:59.999,-0.005322,6.33,0.00,48.02,...,0.004549,58.621292,5.41,0.026613,4907.642855,-0.255094,19.482396,0,0.886660,0.299981
20185,118942.61,118942.62,118927.06,118927.06,1.35905,2025-07-24 03:09:59.999,-0.013074,15.55,0.01,0.00,...,-0.028194,53.576246,-33.54,0.025650,4906.283805,-0.492824,19.262842,0,0.301096,-0.461385
